In [4]:
import os
import json
import torch

print(f"Torch Version : {torch.__version__}")

print(f"CUDA Available : {torch.cuda.is_available()}")

print(f"CUDA Version : {torch.version.cuda}")

dataset = [
    {
        "instruction": "Translate English to Tamil",
        "input" : "Good Morning!",
        "output" : "காலை வணக்கம்!"
    },
    {
        "instruction": "Summarize",
        "input" : "Deep learning enables computers to learn complex patterns.",
        "output" : "Deep learning lets computers learn patterns"
    }
]

os.makedirs("data",exist_ok = True)

with open("data/sample.jsonl","w") as f:
   for ex in dataset:
     f.write(json.dumps(ex) + "\n")


print("Sample Dataset Saved..")




Torch Version : 2.6.0+cu124
CUDA Available : True
CUDA Version : 12.4
Sample Dataset Saved..


In [5]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 771.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.8

In [ ]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer, DataCollatorLanguageModeling

model_name = "tinyllama/tinyllama-1.1b-chat-v1.0"

model, tokenzier = FastLanguageModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = True
)

model = FastLanguageModel.get_peft_model(
    model,
    r= 8,
    target_modules = ["q_proj","v_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True)

dataset = load_dataset("json",data_files="data/sample.jsonl",split="train")


def tokenzier_fn(example) :
   prompt : f"###Instruction: {example['instruction']}\n###Input: {example['input']}\n###Output: {example['output']}"
   tokenized = tokenizer(
       prompt,
       truncation = True,
       max_length = 256,
       padding = "max_length"
   )
   tokenized["labels"] = tokenized["input_ids"].copy()
   return tokenized


tokenized_dataset = dataset.map(tokenzier_fn)

training_args = TrainingArguments(
    output_dir = "output",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 2,
    learning_rate = 2e-4,
    logging_steps=1,
    num_train_epochs=1,
    bf16=torch.cuda.is_bf16_supported(),
    tf32=not torch.cuda.is_bf16_supported(),
    optim = "adamw_torch",
    lr_scheduler_type="cosine",
    report_to="none",
)

data_collator = DataCollatorLanguageModeling(tokenizer = tokenzier, mlm = False)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    data_collator = data_collator
)

trainer.train()
print("Training completed.")